# Workshop Основы LangGraph
## Теория
1. Используемые библиотеки
2. Базовые элементы LangGraph
3. Постановка задачи на создание печатного журнала
## Практика
4. Функции создания разметки журнала
5. Создание контента журнала c помощью GenAI

## Что будет использовано

### 1. LangGraph

✅ Построение сложных NLP-воркфлоу

✅ Ветвление логики выполнения

✅ Интеграция с LLM (OpenAI, Anthropic и др.)

In [ ]:
!pip install -q langgraph

### 2. WeasyPrint

✅ Конвертация веб-контента в PDF

✅ Поддержка современных CSS

✅ Сохранение дизайна без скрытого запуска браузера 

In [ ]:
!pip install -q weasyprint

Для пользователей Windows процесс установки сложнее и описывается по ссылке

https://doc.courtbouillon.org/weasyprint/stable/first_steps.html

Преобразование в PDF не является важным с точки зрения воркшопа, поэтому можно установить значение в следующее ячейке в False, чтобы избежать необязательных затруднений, если weasyprint доставляет неудобства

In [ ]:
should_convert_to_pdf = False

### 3. Horde_SDK

✅ Доступ к бесплатным ИИ-ресурсам

✅ Генерация через crowd-сеть

✅ Поддержка изображений/текста

In [ ]:
!pip install -q horde_sdk

**Регистрация в AI Horde**

Процесс очень простой. Достаточно войти через Google ID и ввести имя. Ключ API будет присвоен сразу. Без регистрации есть доступ с самым низким приоритетом. Чтобы не ждать слишком долго, рекомендуется зарегистрироваться по ссылке
https://stablehorde.net/register

In [ ]:
%%html
<style>
table {float:left}
</style>

# 🧩 Базовые элементы LangGraph

## 1. **Узлы (Nodes)**
```python
from langgraph.graph import Node

# Функция-обработчик узла
def retrieve_data(state):
    return {"data": "Some processed info"}
```
**Назначение**: Базовые строительные блоки графа

**Особенности**:

- Выполняют атомарные операции

- Принимают/возвращают состояние (state)

- Могут быть sync/async

## 2. Состояние (State)
```python
class AgentState(TypedDict):
    input: str
    intermediate_results: list
    final_output: Optional[str]
```
**Роль**: Хранит данные между узлами

**Форматы**:

- Dataclass

- Pydantic-модели

- TypedDict

## 3. Рёбра (Edges)
```python
graph.add_edge("node_a", "node_b")  # Линейное выполнение
```
**Назначение**: Переходы между узлами

## 4. Условные переходы (Conditionals)
```python
def should_continue(state):
    if state["quality"] > 0.7:
        return "accept" 
    return "revise"

graph.add_conditional_edges(
    "evaluator",
    should_continue,
    {"accept": "publish", "revise": "editor"}
)
```
**Использование**:

- Маршрутизация по условиям

- Повторные циклы (retry-логика)

## 5. Граф (Graph)
```python
from langgraph.graph import Graph

workflow = Graph()
workflow.add_node("preprocess", preprocess_data)
workflow.add_node("generate", call_llm)
workflow.add_edge("preprocess", "generate")
```
**Методы управления**:

- .add_node() - добавление узла

- .add_edge() - создание связи

- .add_conditional_edges() - ветвление


## 📌 Ключевые концепции
| Элемент   | Аналог в программировании |
|-----------|---------------------------|
| Узлы	    | Функции                   | 
| Рёбра	    | Вызовы функций            |
| Состояние	| Контекст выполнения       |
| Условия	| if/else логика            |


# Что мы с этим будем делать?
### Построим граф для создания журнала
У журнала должна быть обложка, несколько страниц со статьями и завершающая страница с редакторами. Журнал должен быть иллюстрирован, а статьи содержать комментарии редактора.

# Пишем код!
### Импортируем все необходимое

In [ ]:
import json
import os
from typing import Dict, TypedDict, List
from langgraph.graph import Graph
from IPython.display import Image as IImage, display
from pathlib import Path
from PIL.Image import Image
from horde_sdk import ANON_API_KEY
from horde_sdk.ai_horde_api.ai_horde_clients import AIHordeAPISimpleClient
from horde_sdk.ai_horde_api.apimodels import (
    ImageGenerateAsyncRequest,
    ImageGenerationInputPayload
)

if should_convert_to_pdf:
    from weasyprint import HTML

### В этом файле сохранены HTML-шаблоны для всех элементов журнала

In [ ]:
html_json_path = "2/html.json"

with open(html_json_path) as f:
    html_dict = json.load(f)

### Объявим класс состояния

In [ ]:
# Класс состояния
class MagazineState(TypedDict):
    title: str
    cover_page: str
    articles: List[Dict]  # Список всех статей
    content_pages: List[str]  # Сгенерированные HTML страницы
    editors_page: str
    html_output: str
    filename: str
    pdf_name: str
    publishing_info: str
    publisher: str
    current_article_index: int  # Индекс текущей обрабатываемой статьи
    keep_adding_articles: bool

### Объявим функции, которые станут узлами. Состояние на вход, состояние на выход

In [ ]:
def add_editors_page(state: MagazineState) -> MagazineState:
    """Генерация страницы редакторов"""
    editors_html = ""
    for editor in state.get("editors_data"):
        editors_html += html_dict["editor_template"].format(
            photo_url=editor["photo_url"],
            name=editor["name"],
            position=editor["position"],
            bio=editor["bio"]
        )

    publishing_note = html_dict["publishing_note_template"].format(
        publisher=state["publisher"],
        publishing_info=state["publishing_info"]
    )
    
    state["editors_page"] = html_dict["editors_page_template"].format(
        editors_html=editors_html,
        publishing_note=publishing_note
    )

    return state

def generate_html(state: MagazineState) -> MagazineState:
    """Финальная сборка HTML"""
    state["html_output"] = html_dict["html_template"].format(
        title=state["title"],
        cover_page=state["cover_page"],
        content_pages="\n".join(state["content_pages"]),
        editors_page=state["editors_page"]
    )
    
    return state

def save_to_file(state: MagazineState) -> MagazineState:
    """Сохранение в файл"""
    with open(state.get("filename"), 'w', encoding='utf-8') as f:
        f.write(state["html_output"])
        
    print(f"HTML журнала успешно сгенерирован и сохранен в {state.get('filename')}")
    
    return state

def save_to_pdf(state: MagazineState) -> MagazineState:     
    HTML(filename=state.get("filename")).write_pdf(state.get("pdf_name"))
    print(f"PDF успешно сгенерирован и сохранен в {state.get('pdf_name')}")
    
    return state

def initialize_magazine(state: MagazineState) -> MagazineState:
    """Инициализация журнала"""
    state["current_article_index"] = 0
    state["content_pages"] = []

    return state

def add_cover_page(state: MagazineState) -> MagazineState:
    """Генерация обложки журнала"""
    state["cover_page"] = html_dict["cover_page_template"].format(
        image_url=state.get("cover_image_url"),
        title_line1=state.get("title_line1"),
        title_line2=state.get("title_line2")
    )
    
    return state

# Здесь избыточно, но так бывает не всегда, зато граф визуализируется нагляднее
def should_add_another_article(state: MagazineState) -> MagazineState:
    """Определяет состояние, нужно ли добавлять еще одну статью"""
    state["keep_adding_articles"] = state["current_article_index"] < len(state["articles"])
    return state

def add_content_page(state: MagazineState):
    """Добавление контентной страницы для текущей статьи"""
    article = state["articles"][state["current_article_index"]]

    page = html_dict["content_page_template"].format(
        title=article["title"],
        image_url=article["image_url"],
        image_alt=article["image_alt"],
        text=article["text"],
        editor_photo_url=article["editor_photo_url"],
        editor_comment=article["editor_comment"]
    )    
    state["content_pages"].append(page)
    state["current_article_index"] += 1  # Переходим к следующей статье
    return state

### Объявим функцию для условного перехода. Состояние на вход, строка на выход

In [ ]:
def keep_adding_articles(state: MagazineState) -> str:
    """Определяет роутинг, нужно ли добавлять еще одну статью"""
    return "yes" if state["keep_adding_articles"] else "no"

### Объявим граф и добавим в него узлы

In [ ]:
workflow = Graph()

# Добавляем узлы
workflow.add_node("initialize", initialize_magazine)
workflow.add_node("add_cover", add_cover_page)
workflow.add_node("add_content", add_content_page)
workflow.add_node("should_add_another", should_add_another_article)
workflow.add_node("add_editors", add_editors_page)
workflow.add_node("generate_html", generate_html)
workflow.add_node("save_html_file", save_to_file)

if should_convert_to_pdf:
    workflow.add_node("save_pdf", save_to_pdf)

### Добавим ребра, условный переход, начальную и конечную точки

In [ ]:
# Определяем связи
workflow.add_edge("initialize", "add_cover")
workflow.add_edge("add_cover", "add_content")
workflow.add_edge("add_content", "should_add_another")

# Условный переход: добавляем еще статью или переходим к редакторам
workflow.add_conditional_edges(
    "should_add_another", keep_adding_articles, {"yes": "add_content", "no": "add_editors"}
)

workflow.add_edge("add_editors", "generate_html")
workflow.add_edge("generate_html", "save_html_file")

if should_convert_to_pdf:
    workflow.add_edge("save_html_file", "save_pdf")

workflow.set_entry_point("initialize")

if should_convert_to_pdf:
    workflow.set_finish_point("save_pdf")
else:
    workflow.set_finish_point("save_html_file")

### Соберем граф и визуализируем его

In [ ]:
magazine_agent = workflow.compile()
display(IImage(magazine_agent.get_graph().draw_mermaid_png()))

### Заполним состояние контентом журнала

In [ ]:
input_state = {
    "title": "Новости Средиземья",
    "cover_image_url": "2/cover.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=1350&q=80",
    "title_line1": "Новости Средиземья",
    "title_line2": "Выпуск 285",
    "articles": [
        {
            "title": "Леголас открывает эльфийскую школу стрельбы из лука в Ривенделле",
            "image_url": "2/default.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=1000&q=80",
            "image_alt": "Вместо картинки",
            "text": "Знаменитый эльф Леголас, герой Войны Кольца, объявил о создании первой в Средиземье школы стрельбы из лука для всех рас. Школа примет учеников уже этим летом, и, по словам принца-эльфа, 'Даже хоббиты смогут научиться попадать в цель с пятидесяти шагов'. Поговаривают, что сам Арагорн лично одобрил эту инициативу, а Гимли, хотя и ворчит, что 'Луки — оружие для тех, кто боится честного боя', пообещал провести мастер-класс по владению боевым топором.",
            "editor_photo_url": "2/editor.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=200&q=80",
            "editor_comment": "Редактор не имеет мнения по этому вопросу..."
        },
        {
            "title": "В Мории вновь зажглись огни — гномы возвращаются в Казад-Дум",
            "image_url": "2/default.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=1000&&q=80",
            "image_alt": "Вместо картинки",
            "text": "После долгих лет очищения от орков и теней прошлого гномы клана Дьюрина начали масштабное заселение древнего королевства Казад-Дум. Король Гимли возглавляет экспедицию, и уже первые отряды строителей восстановили часть великих залов. 'Мория снова будет сиять, как в дни былой славы!' — заявил Гимли, демонстрируя первые слитки митрила, добытые в обновлённых шахтах. Однако рудокопы советуют не углубляться в нижние уровни — ходят слухи, что в тенистых углах ещё могут скрываться остатки древних ужасов.",
            "editor_photo_url": "2/editor.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=200&q=80",
            "editor_comment": "Редактор не имеет мнения по этому вопросу..."
        }
    ],
    "editors_data" : [
        {
            "name": "ИИ Ежов",
            "position": "Главный редактор",
            "photo_url": "2/editor.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=300&q=80",
            "bio": "Основатель журнала, признанный эксперт в области критики, мизантроп, визионер, криптовалютный инвестор,  лучший редактор России."
        },
        {
            "name": "Вадим Белов",
            "position": "Помощник главного редактора",
            "photo_url": "2/vb.jpg?ixlib=rb-1.2.1&auto=format&fit=crop&w=300&q=80",
            "bio": "Уверенный пользователь ПК."
        }
    ],
    "publisher": "Издательство 'Назгул сотоварищи'",
    "publishing_info": "Четвертая эпоха, год 10-й",
    "filename": "magazine_article.html",
    "pdf_name": "magazine_article.pdf"
}


In [ ]:
result = magazine_agent.invoke(input_state)
print(f"Журнал с {len(input_state['articles'])} статьями сохранен в: {result['pdf_name'] if should_convert_to_pdf else result['filename']}")

# Верстка журнала автоматизирована. Теперь автоматизируем создание контента

In [ ]:
def simple_generate_example(prompt: str, api_key: str = ANON_API_KEY) -> str:
    """ Генерирует изображение по промпту и возвращает путь, по которому оно сохранено """
    simple_client = AIHordeAPISimpleClient()

    status_response, job_id = simple_client.image_generate_request(
        ImageGenerateAsyncRequest(
            apikey=api_key,
            params=ImageGenerationInputPayload(
                width=512,
                height=512,
            ),
            prompt=prompt,
            models=["Deliberate"]
        )
    )

    if len(status_response.generations) == 0:
        raise ValueError("No generations returned in the response.")

    example_path = Path("requested_images")
    example_path.mkdir(exist_ok=True, parents=True)
    img_path = f"{example_path}/{job_id}.png"

    image_pil = simple_client.download_image_from_generation(status_response.generations[0])
    image_pil.save(img_path)
    print(f"Image saved to {img_path}")
        
    return img_path

### Нам также понадобится обращаться к LLM для генерации текста. Воспользуемся Mistral из прошлого воркшопа
Полный список нативно поддерживаемых провайдеров представлен здесь https://python.langchain.com/docs/integrations/llms/

Есть даже GigaChat.

### Установим нужный пакет

In [ ]:
!pip install -q langchain_mistralai

### Установим ключ доступа

In [ ]:
os.environ['MISTRAL_API_KEY'] = "<your-api-key>"

### И создадим нужный класс

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")

### Напишем функцию генерации комментария

In [ ]:
def generate_comment(title: str, text: str) -> str:
    """Создает комментарий редактора по заголовку и тексту новости"""
    messages = [
        (
            "system",
            "Ты редактор журнала и пишешь к статьям ироничные комментарии, которые никого не оскорбляют.",
        ),
        (
            "human", 
            f"Напиши комментарий (только комментарий) к статье с заголовком '{title}' и следующим текстом: {text}"
        ),
    ]
    ai_msg = llm.invoke(messages)
    
    return ai_msg.content

### Нам понадобится новый узел для графа

In [ ]:
def add_editors_comments(state: MagazineState) -> MagazineState:
    """Генерирует комментарии редактора"""
    for article_index in range(len(state["articles"])):  
        comment = generate_comment(state["articles"][article_index]["title"], state["articles"][article_index]["text"])
        state["articles"][article_index]["editor_comment"] = comment
    
    return state

### Нам понадобится возможность генерации картинок на лету
На 11.04.2025 возможность бесплатно генерировать изображения по API предоставляется очень малвм числом сервисов. Мы возпользуемся AI Horde

In [ ]:
def generate_image(prompt: str, api_key: str = ANON_API_KEY) -> str:
    """ Генерирует изображение по промпту и возвращает путь, по которому оно сохранено """
    simple_client = AIHordeAPISimpleClient()

    status_response, job_id = simple_client.image_generate_request(
        ImageGenerateAsyncRequest(
            apikey=api_key,
            params=ImageGenerationInputPayload(
                width=512,
                height=256,
            ),
            prompt=prompt,
            models=["Deliberate"]
        )
    )

    if len(status_response.generations) == 0:
        raise ValueError("No generations returned in the response.")

    example_path = Path("requested_images")
    example_path.mkdir(exist_ok=True, parents=True)
    img_path = f"{example_path}/{job_id}.png"

    image_pil = simple_client.download_image_from_generation(status_response.generations[0])
    image_pil.save(img_path)
    print(f"Image saved to {img_path}")
        
    return img_path

### Вставим ключ для AI Horde

In [ ]:
horde_api_key = "<your-api-key>" # ANON_API_KEY

### Но какую картинку генерировать? А это тоже пускай решает LLM

In [ ]:
def generate_image_description(title: str, text: str) -> str:
    """Генерирует описание иллюстрации"""
    messages = [
        (
            "system",
            "Ты редактор журнала и придумываешь на английском языке, какие иллюстрации подходят новостям.",
        ),
        (
            "human", 
            f"Напиши на английском языке короткое описание (только описание) иллюстрации к статье с заголовком '{title}' и следующим текстом: {text}"
        ),
    ]
    ai_msg = llm.invoke(messages)
    
    return ai_msg.content

### Добавление иллюстраций тоже сделаем отдельным узлом

In [ ]:
def add_article_images(state: MagazineState) -> MagazineState:
    """Генерирует иллюстрации к статьям"""
    for article_index in range(len(state["articles"])):  
        image_desc = generate_image_description(state["articles"][article_index]["title"], state["articles"][article_index]["text"])
        state["articles"][article_index]["image_alt"] = image_desc
        image_path = generate_image(image_desc, horde_api_key)
        state["articles"][article_index]["image_url"] = image_path
    
    return state

### Снова создадим граф с учетом новых узлов и ребер

In [ ]:
workflow = Graph()
workflow.add_node("initialize", initialize_magazine)
workflow.add_node("add_cover", add_cover_page)
workflow.add_node("add_content", add_content_page)
workflow.add_node("should_add_another", should_add_another_article)
workflow.add_node("add_editors", add_editors_page)
workflow.add_node("generate_html", generate_html)
workflow.add_node("save_html_file", save_to_file)

if should_convert_to_pdf:
    workflow.add_node("save_pdf", save_to_pdf)
    
# Новые узлы
workflow.add_node("add_comments", add_editors_comments)
workflow.add_node("add_images_to_articles", add_article_images)

workflow.add_edge("initialize", "add_cover")
# Новые ребра
workflow.add_edge("add_cover", "add_comments")
workflow.add_edge("add_comments", "add_images_to_articles")
workflow.add_edge("add_images_to_articles", "add_content")

workflow.add_edge("add_content", "should_add_another")
workflow.add_conditional_edges(
    "should_add_another", keep_adding_articles, {"yes": "add_content", "no": "add_editors"}
)
workflow.add_edge("add_editors", "generate_html")
workflow.add_edge("generate_html", "save_html_file")

if should_convert_to_pdf:
    workflow.add_edge("save_html_file", "save_pdf")

workflow.set_entry_point("initialize")

if should_convert_to_pdf:
    workflow.set_finish_point("save_pdf")
else:
    workflow.set_finish_point("save_html_file")

### Новый граф стал больше

In [ ]:
magazine_agent = workflow.compile()
display(IImage(magazine_agent.get_graph().draw_mermaid_png()))

### Осталось только вылнить граф заново

In [ ]:
result = magazine_agent.invoke(input_state)
print(f"Журнал с {len(input_state['articles'])} статьями сохранен в: {result['pdf_name'] if should_convert_to_pdf else result['filename']}")

## Вместо выводов
Можно ли было сделать все то же самое без LangGraph? Можно.

Можно ли без LangGraph сделать код элегантнее? Можно.

Но это только базовые элементы, без изучения которых нельзя продвинуться дальше. В следующих воркшопах будут продемонстрированы те возможности LangGraph, которые делают его одним из лидеров рынка фреймворков AI агентов.

## Что дальше
   - Изучаем LangGraph глубже на бесплатном курсе от разработчиков https://academy.langchain.com/courses/intro-to-langgraph
   - Ждем следующих воркшопов от  <img src="https://telegram.org/img/t_logo.png" width="16" height="16">@ds_professional
